In [5]:
! pwd

/content


In [8]:
! unzip brain.zip

Archive:  brain.zip
   creating: brain/
  inflating: brain/1.dcm             
  inflating: brain/10.dcm            
  inflating: brain/100.dcm           
  inflating: brain/101.dcm           
  inflating: brain/102.dcm           
  inflating: brain/103.dcm           
  inflating: brain/104.dcm           
  inflating: brain/105.dcm           
  inflating: brain/106.dcm           
  inflating: brain/107.dcm           
  inflating: brain/108.dcm           
  inflating: brain/109.dcm           
  inflating: brain/11.dcm            
  inflating: brain/110.dcm           
  inflating: brain/111.dcm           
  inflating: brain/112.dcm           
  inflating: brain/113.dcm           
  inflating: brain/114.dcm           
  inflating: brain/115.dcm           
  inflating: brain/116.dcm           
  inflating: brain/117.dcm           
  inflating: brain/118.dcm           
  inflating: brain/119.dcm           
  inflating: brain/12.dcm            
  inflating: brain/120.dcm           
  inflatin

In [1]:
! pip install SimpleITK pydicom nibabel numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.2 MB/s eta 0:00:00


In [11]:
import os
import numpy as np
import pydicom
import SimpleITK as sitk
import nibabel as nib

def get_series_with_max_slices(dicom_dir):
    reader = sitk.ImageSeriesReader()
    series_ids = reader.GetGDCMSeriesIDs(dicom_dir)
    if not series_ids:
        raise Exception("No DICOM series found.")

    max_len = 0
    selected_series = None

    for series_id in series_ids:
        files = reader.GetGDCMSeriesFileNames(dicom_dir, series_id)
        if len(files) > max_len:
            max_len = len(files)
            selected_series = series_id

    return selected_series, reader.GetGDCMSeriesFileNames(dicom_dir, selected_series)

def load_hu_volume(dicom_files):
    slices = [pydicom.dcmread(f) for f in dicom_files]
    slices.sort(key=lambda s: float(s.ImagePositionPatient[2]))

    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.float32)


    slopes = [float(s.RescaleSlope) for s in slices]
    intercepts = [float(s.RescaleIntercept) for s in slices]

    if not all(s == slopes[0] for s in slopes):
        print("Warning: Different slopes across slices.")
    if not all(i == intercepts[0] for i in intercepts):
        print("Warning: Different intercepts across slices.")

    hu_volume = image * slopes[0] + intercepts[0]
    return hu_volume

def save_nifti_from_dicom(dicom_files, output_path):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(dicom_files)
    image = reader.Execute()
    sitk.WriteImage(image, output_path)
    print(f"NIfTI saved to: {output_path}")

def compare_volumes(vol1, vol2):
    if vol1.shape != vol2.shape:
        print("Shape mismatch")
        return

    diff = np.abs(vol1 - vol2)
    max_diff = np.max(diff)
    mean_diff = np.mean(diff)

    print(f"Max difference: {max_diff:.4f}")
    print(f"Mean difference: {mean_diff:.4f}")
    if max_diff < 1e-3:
        print("HU volumes match!")
    else:
        print("HU volumes do not match exactly.")

def compare_volumes_verbose(vol1, vol2, tolerance=1e-3):
    if vol1.shape != vol2.shape:
        print("Shape mismatch")
        return

    exact_match_matrix = vol1 == vol2
    close_match_matrix = np.isclose(vol1, vol2, atol=tolerance)

    print(f"Shape match: {vol1.shape}")

    print(f"\nExact match matrix (vol1 == vol2):")
    print(exact_match_matrix.astype(int))

    print(f"\nClose match matrix (tolerance ±{tolerance}):")
    print(close_match_matrix.astype(int))

    percent_match = np.sum(close_match_matrix) / close_match_matrix.size * 100
    print(f"\nPercentage of values matching within tolerance: {percent_match:.2f}%")

    max_diff = np.max(np.abs(vol1 - vol2))
    mean_diff = np.mean(np.abs(vol1 - vol2))
    print(f"\nMax absolute difference: {max_diff:.4f}")
    print(f"Mean absolute difference: {mean_diff:.4f}")



# === SET THIS PATH ===
dicom_folder = "/content/brain"
output_nifti_path = "/content/brain_last.nii.gz"

# === RUN WORKFLOW ===
series_id, dicom_files = get_series_with_max_slices(dicom_folder)
print(f"Using Series ID: {series_id}")
print(f"Number of DICOM slices: {len(dicom_files)}")

hu_volume_original = load_hu_volume(dicom_files)
print(f"HU volume shape: {hu_volume_original.shape}")

save_nifti_from_dicom(dicom_files, output_nifti_path)

# Load from NIfTI and compare
sitk_image = sitk.ReadImage(output_nifti_path)
hu_volume_nifti = sitk.GetArrayFromImage(sitk_image)

compare_volumes(hu_volume_original, hu_volume_nifti)
compare_volumes_verbose(hu_volume_original, hu_volume_nifti)


Using Series ID: 1.3.12.2.1107.5.1.4.73214.30000021012203105829900046849
Number of DICOM slices: 133
HU volume shape: (133, 512, 512)
NIfTI saved to: /content/brain_last.nii.gz
Max difference: 0.0000
Mean difference: 0.0000
HU volumes match!
Shape match: (133, 512, 512)

Exact match matrix (vol1 == vol2):
[[[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  ...
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]]

 [[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  ...
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]]

 [[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  ...
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]]

 ...

 [[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  ...
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]]

 [[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  ...
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]]

 [[1 1 1 ... 1 1 1]
  [1 1 1 ... 1 1 1]
  [1 1 